In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
from talib import abstract
import math

plt.style.use('fivethirtyeight')

In [2]:
data = pd.read_csv('data.csv')
# data = data.iloc[-40880:-1]
# data = data.reset_index()
# dataframe

In [3]:
data['sma50'] = abstract.SMA(data, timeperiod=50)
data['sma20'] = abstract.SMA(data, timeperiod=20)
# data['sma150'] = abstract.SMA(data, timeperiod=150)

In [4]:
money = 1000
sigPriceBuy = []
sigPriceSell = []

earns = pd.DataFrame(data = {
    'buy time': [],
    'buy': [],
    'sell': [],
    'sell time': [],
    'earn': [],
})

flag = 'sold'

for i in range(len(data)):
    if i <= 150:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
        continue

    #SELL
    if flag == 'bought':
        last_40_prices = data.iloc[i - 41: i - 1]['close']
        higher_price = max(last_40_prices)
        
        rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
        
        if data['close'][i] > higher_price:
            
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(data['close'][i])
            flag = 'sold'
            earns.loc[rowIndex, 'sell'] = data['close'][i]
            earns.loc[rowIndex, 'earn'] = data['close'][i] - earns.loc[rowIndex, 'buy'] 
            earns.loc[rowIndex, 'sell time'] = data['open time'][i]
            continue
            
        variation = (data['close'][i] / earns['buy'][rowIndex]) - 1
            
        if variation <= -0.01:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(data['close'][i])
            flag = 'sold'
            earns.loc[rowIndex, 'sell'] = data['close'][i]
            earns.loc[rowIndex, 'earn'] = data['close'][i] - earns.loc[rowIndex, 'buy'] 
            earns.loc[rowIndex, 'sell time'] = data['open time'][i]
            continue
        
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
        continue





    # BUY                 
    initial_value = data.iloc[i-150]
    current_value = data.iloc[i]

    is_price_going_down = initial_value['close'] >= current_value['close']

    if is_price_going_down:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
        continue

    tangent = (data['sma150'][i] - data['sma150'][i-150]) / 150 
    tendency_degrees = math.degrees(math.atan(tangent))
    is_bull_tendency = tendency_degrees > 1

    if not is_bull_tendency:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
        continue
    
    
    if current_value['close'] <= data['sma50'][i]:
        if flag == 'sold':
            sigPriceBuy.append(data['close'][i])
            sigPriceSell.append(np.nan)

            flag = 'bought'
            earns = earns.append({ 
                'buy': data['close'][i],
                'buy time': data['open time'][i]
            }, ignore_index=True)
            continue


    sigPriceBuy.append(np.nan)
    sigPriceSell.append(np.nan)

In [5]:
print('Lucro', earns['earn'].sum())
total = earns['earn'].count()
print('Quantidade', earns['earn'].count())
wins = earns[earns['earn'] > 0].count()
p = (wins/total)['earn'] * 100
print(f'Porcentagem {p}%')
print('-------')

Lucro 2.6000000000000227
Quantidade 2
Porcentagem 100.0%
-------


In [6]:
# df = dataframe.iloc[0:10]
# print(df)
# plt.figure(figsize = (50, 12.5))
# plt.plot(df['open time'], df['close'], label = 'BNB/BTC', alpha=0.30)
# plt.plot(df['open time'], df['sma3'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma9'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma20'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma150'], label = 'SMA 100', alpha=0.30)
# plt.scatter(df['open time'], df['buy'], label = 'Buy', marker = '^', color = 'green')
# plt.scatter(df['open time'], df['sell'], label = 'Sell', marker = 'v', color = 'red')
# plt.title('BNB/BTC Close price')
# plt.xlabel('Date')
# plt.ylabel('Close price')
# # plt.legend(loc='upper left')
# plt.show()

In [7]:
earns = earns[earns['buy'].notna() | earns['sell'].notna()]
earns.to_csv("result.csv", index=False)